# Laboratorio 2 - Complejidad y búsqueda de hiperparámetros
### Sergio David Ferreira - Juan David Gutierrez


## Librerías utilizadas

In [ ]:
import random
import pandas as pd
import numpy as np
from sklearn import set_config
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, FunctionTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler, PolynomialFeatures
from sklearn.compose import ColumnTransformer


## Carga de datos y partición de datos
Para la carga de datos se carga el archivo del laboratorio anterior, se eliminan pacientes duplicados y outliers y valores faltantes sobre la columna target, *CVD Risk Score*

In [77]:
data = pd.read_csv('data/Datos Lab 1.csv')

# Normalizamos fechas
data['Date of Service'] = pd.to_datetime(data['Date of Service'], format='mixed')

# Eliminamos pacientes con registros duplicados manteniendo el más reciente
data = data.sort_values(['Patient ID', 'Date of Service'], ascending=[True, False])
data = data.drop_duplicates(subset='Patient ID', keep='first')

# Eliminamos registros con valores faltantes en el target
data = data.dropna(subset=['CVD Risk Score'])

# Eliminamos registros de outliers en la variable target
q1 = data['CVD Risk Score'].quantile(0.25)
q3 = data['CVD Risk Score'].quantile(0.75)
iqr = q3 - q1

limite_inferior = q1 - 1.5 * iqr
limite_superior = q3 + 1.5 * iqr

data = data[(data['CVD Risk Score'] > limite_inferior) & (data['CVD Risk Score'] < limite_superior)]

#Mostramos un sample de los datos
data.sample(3)

,Patient ID,Date of Service,Sex,Age,Weight (kg),Height (m),BMI,Abdominal Circumference (cm),Blood Pressure (mmHg),Total Cholesterol (mg/dL),...,Physical Activity Level,Family History of CVD,Height (cm),Waist-to-Height Ratio,Systolic BP,Diastolic BP,Blood Pressure Category,Estimated LDL (mg/dL),CVD Risk Score,CVD Risk Level
869,LcWw5698,2024-02-16,M,43.0,73.500,1.620,28.000,86.700,111/87,200.0,...,High,N,162.00,0.535,111.0,87.0,Hypertension Stage 1,108.0,15.150,LOW
48,FTEC4446,2022-11-05,F,73.0,66.074,2.146,27.461,76.599,112/106,197.0,...,Low,Y,195.82,0.391,112.0,106.0,Hypertension Stage 2,112.0,15.325,INTERMEDIARY
93,GjTt5411,2020-02-16,M,37.0,79.300,1.860,22.900,73.800,118/89,286.0,...,Low,N,186.00,0.397,118.0,89.0,Hypertension Stage 1,NaN,18.200,HIGH


Ahora, separamos los datos en entrenamiento y prueba

In [78]:
target = 'CVD Risk Score'
X = data.drop(columns=[target])
y = data[target]

In [79]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=1)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_train.shape)

(1038, 23) (1038,)
(260, 23) (1038,)


## Construcción del pipeline

Para la construcción del pipeline se sigue el procedimiento del laboratorio anterior, por lo tanto únicamente se enuncian los pasos sin añadir justificaciones del proceso.

### Codificación de variables categóricas

In [80]:
numeric_features = [
    "Age", "Weight (kg)", "Height (m)", "Height (cm)", "BMI", "Abdominal Circumference (cm)", 
    "Total Cholesterol (mg/dL)", "HDL (mg/dL)", "Fasting Blood Sugar (mg/dL)", 
    "Waist-to-Height Ratio", "Systolic BP", "Diastolic BP", "Estimated LDL (mg/dL)"
]

ordinal_features = [
    "Physical Activity Level", "Blood Pressure Category"
]

binary_features = [
    "Sex", "Smoking Status", "Diabetes Status", "Family History of CVD"
]

binary_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(drop='if_binary', handle_unknown='ignore')),
    ('imputer', SimpleImputer(strategy=lambda x: random.randint(0, 1)))
])

ordinal_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ordinal', OrdinalEncoder(categories = [
        ['Low', 'Moderate', 'High'],  # Physical Activity Level
        ['Normal', 'Elevated', 'Hypertension Stage 1', 'Hypertension Stage 2']  # Blood Pressure Category
    ]))
])


### Eliminación de columnas redundantes o innecesarias

In [81]:
col_drop = ['Blood Pressure (mmHg)', 'Patient ID', 'Date of Service', 'CVD Risk Level']
def drop_columns(df):
    return df.drop(columns=col_drop, errors='ignore')

dropper = FunctionTransformer(drop_columns)

### Manejo de outliers por rangos válidos de las variables

In [82]:
rangos = {
    'Diastolic BP': (40, 125),
    'Systolic BP': (80, 190),
    'Fasting Blood Sugar (mg/dL)': (60, 200),
    'Total Cholesterol (mg/dL)': (80, 320),
    'HDL (mg/dL)': (20, 100),
    'Estimated LDL (mg/dL)': (0, 250),
    'BMI': (15, 50),
    'Weight (kg)': (30, 140),
    'Height (m)': (1.45, 2),
    'Height (cm)': (145, 200),
    'Abdominal Circumference (cm)': (60, 130),
    'Waist-to-Height Ratio': (0.35, 0.8),
    'Age': (18, 100)
}
def limpiar_rangos(df):
    for col, (min_val, max_val) in rangos.items():
        if col in df.columns:
            df.loc[
                (df[col] <= min_val) | (df[col] >= max_val),
                col
            ] = np.nan
    return df
rangos_validos = FunctionTransformer(limpiar_rangos)
numeric_transformer = Pipeline(steps=[
    ('rangos', rangos_validos)
])

### Imputación de valores faltantes
Variables sin multicolinealidad

In [83]:
clean_numeric_data = numeric_transformer.fit_transform(data)
clean_numeric_data = pd.DataFrame(clean_numeric_data, columns=numeric_features)

median_columns = ['Age', 'Diastolic BP', 'Systolic BP', 'Fasting Blood Sugar (mg/dL)', 'HDL (mg/dL)']
mean_columns = ['Weight (kg)', 'BMI']

def median_transformer_func(X):
    for col in median_columns:
        X[col] = X[col].fillna(clean_numeric_data[col].median())
    return X

def mean_transformer_func(X):
    for col in mean_columns:
        X[col] = X[col].fillna(clean_numeric_data[col].mean())
    return X

median_transformer = FunctionTransformer(median_transformer_func)
mean_transformer = FunctionTransformer(mean_transformer_func)
simple_imputer_transformer = ColumnTransformer(transformers=[
    ('median', median_transformer, median_columns),
    ('mean', mean_transformer, mean_columns)
])

Variables con alta colinealidad

In [84]:
# Waist to height ratio
temp_abdominal = clean_numeric_data['Abdominal Circumference (cm)'].fillna(clean_numeric_data['Abdominal Circumference (cm)'].mean())
k_waist_to_height_ratio = (clean_numeric_data['Waist-to-Height Ratio'] / temp_abdominal).dropna().median()

# Abdominal circumference
k_abdominal_circumference = (clean_numeric_data['Abdominal Circumference (cm)'] / clean_numeric_data['Waist-to-Height Ratio']).dropna().median()

# Total cholesterol
temp_ldl = clean_numeric_data['Estimated LDL (mg/dL)'].fillna(clean_numeric_data['Estimated LDL (mg/dL)'].mean())
k_total_cholesterol = (clean_numeric_data['Total Cholesterol (mg/dL)'] / temp_ldl).dropna().median()

# Estimated LDL
k_estimated_ldl = (clean_numeric_data['Estimated LDL (mg/dL)'] / clean_numeric_data['Total Cholesterol (mg/dL)']).dropna().median()

# Height (cm)
temp_height_m = clean_numeric_data['Height (m)'].fillna(clean_numeric_data['Height (m)'].mean())

def correlated_imputer(df):
    df['Waist-to-Height Ratio'] = df['Waist-to-Height Ratio'].fillna(k_waist_to_height_ratio * temp_abdominal)
    df['Abdominal Circumference (cm)'] = df['Abdominal Circumference (cm)'].fillna(k_abdominal_circumference * df['Waist-to-Height Ratio'])
    df['Total Cholesterol (mg/dL)'] = df['Total Cholesterol (mg/dL)'].fillna(k_total_cholesterol * temp_ldl)
    df['Estimated LDL (mg/dL)'] = df['Estimated LDL (mg/dL)'].fillna(k_estimated_ldl * df['Total Cholesterol (mg/dL)'])
    df['Height (cm)'] = df['Height (cm)'].fillna(temp_height_m * 100)
    df['Height (m)'] = df['Height (m)'].fillna(df['Height (cm)'] / 100)
    return df

corr_imputer = FunctionTransformer(correlated_imputer)
corr_features = ['Waist-to-Height Ratio', 'Abdominal Circumference (cm)', 'Total Cholesterol (mg/dL)', 'Estimated LDL (mg/dL)', 'Height (cm)', 'Height (m)']
imputer_transformer = ColumnTransformer(transformers=[
    ('simple', simple_imputer_transformer, median_columns + mean_columns),
    ('correlated', corr_imputer, corr_features)
])
numeric_transformer.steps.append(('imputer', imputer_transformer))

### Eliminación de variables con alta multicolinealidad

In [85]:
columns_vif=['Height (cm)', 'Waist-to-Height Ratio', 'Estimated LDL (mg/dL)', 'Blood Pressure Category']
def drop_high_vif(df):
    f_names = median_columns + mean_columns + corr_features
    df = pd.DataFrame(df, columns=f_names)
    return df.drop(columns=columns_vif, errors='ignore')
dropper_vif = FunctionTransformer(drop_high_vif)
numeric_transformer.steps.append(('dropper_vif', dropper_vif))

### Escalado MinMax

In [86]:
numeric_transformer.steps.append(('scaler', MinMaxScaler()))

### Construcción del pipeline final

In [87]:
set_config(display="diagram")
numeric_transformer

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('rangos', ...), ('imputer', ...), ...]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"func func: callable, default=NoneThe callable to use for the transformation. This will be passedthe same arguments as transform, with args and kwargs forwarded.If func is None, then func will be the identity function.",<function lim...001D11739A400>
,"inverse_func inverse_func: callable, default=NoneThe callable to use for the inverse transformation. This will bepassed the same arguments as inverse transform, with args andkwargs forwarded. If inverse_func is None, then inverse_funcwill be the identity function.",None
,"validate validate: bool, default=FalseIndicate that the input X array should be checked before calling``func``. The possibilities are:- If False, there is no input validation.- If True, then X will be converted to a 2-dimensional NumPy array or sparse matrix. If the conversion is not possible an exception is raised... versionchanged:: 0.22 The default of ``validate`` changed from True to False.",False
,"accept_sparse accept_sparse: bool, default=FalseIndicate that func accepts a sparse matrix as input. If validate isFalse, this has no effect. Otherwise, if accept_sparse is false,sparse matrix inputs will cause an exception to be raised.",False
,"check_inverse check_inverse: bool, default=TrueWhether to check that or ``func`` followed by ``inverse_func`` leads tothe original inputs. It can be used for a sanity check, raising awarning when the condition is not fulfilled... versionadded:: 0.20",True
,"feature_names_out feature_names_out: callable, 'one-to-one' or None, default=NoneDetermines the list of feature names that will be returned by the`get_feature_names_out` method. If it is 'one-to-one', then the outputfeature names will be equal to the input feature names. If it is acallable, then it must take two positional arguments: this`FunctionTransformer` (`self`) and an array-like of input feature names(`input_features`). It must return an array-like of output featurenames. The `get_feature_names_out` method is only defined if`feature_names_out` is not None.See ``get_feature_names_out`` for more details... versionadded:: 1.1",None
,"kw_args kw_args: dict, default=NoneDictionary of additional keyword arguments to pass

## Comparación y selección de modelos
Para evaluar distintos modelos de regresión polinomial aplicamos transformación de variables polinomiales al transformador numérico, para después ajustarlo a un modelo de regresión lineal. Evaluaremos modelos de hasta n grado.

In [ ]:
pipelines = []
for deg in range(1, 4):
    numeric_transformer.steps.append(('polinomial', PolynomialFeatures(degree=deg)))
    preprocessing_transformer = ColumnTransformer(transformers=[
        ('binary', binary_transformer, binary_features),
        ('ordinal', ordinal_transformer, ordinal_features),
        ('numeric', numeric_transformer, numeric_features)
    ])
    pipeline_regresion = Pipeline(steps=[
        ('dropper', dropper),
        ('preprocesamiento', preprocessing_transformer),
        ('modelo', LinearRegression())
    ])
    pipelines.append(pipeline_regresion)